<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense

# Simulasi Data (Data asli harus dikumpulkan terlebih dahulu)
data = {
    'User_ID': [1, 2, 3, 4, 5],
    'Usia': [25, 34, 28, 45, 33],
    'Jenis Kelamin': ['Laki-laki', 'Perempuan', 'Laki-laki', 'Perempuan', 'Laki-laki'],
    'Pendidikan': ['Sarjana', 'Diploma', 'SMA', 'Sarjana', 'Pascasarjana'],
    'Pekerjaan': ['PNS', 'Pekerja Kantoran', 'Wirausahawan', 'PNS', 'Pekerja Kantoran'],
    'Pendapatan': [7000000, 8000000, 5000000, 10000000, 9000000],
    'Bali': [4, 5, 4, 3, 5],
    'Yogyakarta': [5, 4, 3, 4, 5],
    'Lombok': [4, 5, 4, 2, 4],
    'Jakarta': [3, 4, 2, 3, 4],
    'Raja Ampat': [5, 5, 5, 4, 5]
}

df = pd.DataFrame(data)

# Preprocessing
le = LabelEncoder()
df['Jenis Kelamin'] = le.fit_transform(df['Jenis Kelamin'])
df['Pendidikan'] = le.fit_transform(df['Pendidikan'])
df['Pekerjaan'] = le.fit_transform(df['Pekerjaan'])

scaler = MinMaxScaler()
df[['Usia', 'Pendapatan']] = scaler.fit_transform(df[['Usia', 'Pendapatan']])

# Preparing the data for NCF
user_features = df[['User_ID', 'Usia', 'Jenis Kelamin', 'Pendidikan', 'Pekerjaan', 'Pendapatan']]
ratings = df[['Bali', 'Yogyakarta', 'Lombok', 'Jakarta', 'Raja Ampat']]

# Reshape data for training
n_users = user_features.shape[0]
n_items = ratings.shape[1]
user_ids = np.array([[i]*n_items for i in range(n_users)]).flatten()
item_ids = np.array([list(range(n_items))]*n_users).flatten()
ratings_flat = ratings.values.flatten()

# Model Architecture
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=n_users, output_dim=50)(user_input)
item_embedding = Embedding(input_dim=n_items, output_dim=50)(item_input)

user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

input_vecs = Concatenate()([user_vecs, item_vecs])

dense = Dense(128, activation='relu')(input_vecs)
output = Dense(1)(dense)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mse')

# Training the model
model.fit([user_ids, item_ids], ratings_flat, epochs=10, verbose=1)

# Predicting ratings
predictions = model.predict([user_ids, item_ids]).flatten()
predictions = predictions.reshape((n_users, n_items))

# Evaluasi
true_ratings = ratings.values
mae = mean_absolute_error(true_ratings, predictions)
rmse = mean_squared_error(true_ratings, predictions, squared=False)

# Untuk metrik classification, kita perlu binarize ratings
threshold = 3.5
true_binary = (true_ratings > threshold).astype(int)
pred_binary = (predictions > threshold).astype(int)

accuracy = accuracy_score(true_binary.flatten(), pred_binary.flatten())
precision = precision_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
recall = recall_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
f1 = f1_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')

# Output hasil evaluasi
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 17.3744
Epoch 2/10
1/1 [==============================] - 0s 27ms/step - loss: 17.2001
Epoch 3/10
1/1 [==============================] - 0s 28ms/step - loss: 17.0317
Epoch 4/10
1/1 [==============================] - 0s 39ms/step - loss: 16.8676
Epoch 5/10
1/1 [==============================] - 0s 41ms/step - loss: 16.7058
Epoch 6/10
1/1 [==============================] - 0s 40ms/step - loss: 16.5441
Epoch 7/10
1/1 [==============================] - 0s 29ms/step - loss: 16.3820
Epoch 8/10
1/1 [==============================] - 0s 34ms/step - loss: 16.2197
Epoch 9/10
1/1 [==============================] - 0s 41ms/step - loss: 16.0571
Epoch 10/10
1/1 [==============================] - 1s 551ms/step
MAE: 3.8568649154901506
RMSE: 3.9310884920787217
Accuracy: 0.24
Precision: 0.0576
Recall: 0.24
F1-Score: 0.0929032258064516


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
